In [ ]:
import importlib
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from yolo.constants import *

keras_yolo = importlib.import_module('yolo.vendor.keras-yolo3.yolo3_one_file_to_detect_them_all')
bbox = importlib.import_module('yolo.vendor.keras-yolo3.utils.bbox')
utils = importlib.import_module('yolo.vendor.keras-yolo3.utils.utils')

In [ ]:
# define model and photo
photo_path = os.path.join(DIR_DATA, 'images/traffic.jpg')
model_path = os.path.join(DIR_MODELS, 'MSCOCO/model_yolov3.h5')

# Load the model

In [ ]:
# load yolov3 model
model = load_model(model_path)
# define the expected input shape for the model
INPUT_W, INPUT_H = 416, 416

# Load the image to detect bounding boxes for

In [ ]:
def load_image_pixels(filename, shape):
    """Loads an image, reshapes to the expected input shape
    and normalizes the pixel values.
    """
    # load the image to get its shape
    image = load_img(filename)
    width, height = image.size
    # load the image with the required size
    image = load_img(filename, target_size=shape)
    # convert to numpy array
    image = img_to_array(image)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    # add a dimension so that we have one sample
    image = np.expand_dims(image, 0)
    return image, width, height

In [ ]:
# load and prepare image
image, image_w, image_h = load_image_pixels(photo_path, (INPUT_W, INPUT_H))
# make prediction
yhat = model.predict(image)
# summarize the shape of the list of arrays
print([a.shape for a in yhat])

# show the rescaled image
plt.imshow(image[0,...]);

# Logic to decode the YOLO model output and visualize results

In [ ]:
# get all of the results above a threshold
def get_boxes(boxes, labels, thresh):
    """Filters boxes based on their scores. If the score for a particular
    box and class is lower than thresh, then drop the box. Otherwise keep
    it.
    """
    v_boxes, v_labels, v_scores = list(), list(), list()
    # enumerate all boxes
    for box in boxes:
        # enumerate all possible labels
        for i in range(len(MSCOCO_LABELS)):
            # check if the threshold for this label is high enough
            if box.classes[i] > thresh:
                v_boxes.append(box)
                v_labels.append(MSCOCO_LABELS[i])
                v_scores.append(box.classes[i]*100)
                # don't break, many labels may trigger for one box
    return v_boxes, v_labels, v_scores


def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
    """Translates the predicted bounding boxes back to the original
    image size.
    """
    new_w, new_h = net_w, net_h
    for i in range(len(boxes)):
        x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)


def draw_boxes(filename, v_boxes, v_labels, v_scores):
    """Shows the image and plots the bounding boxes.
    """
    # load the image
    data = plt.imread(filename)
    h, w, _ = data.shape
    fig_w = 20
    fig_h = fig_w / w * h
    # plot the image
    fig, ax = plt.subplots(figsize=(fig_h, fig_w))
    
    ax.imshow(data)
    # plot each box
    for i in range(len(v_boxes)):
        box = v_boxes[i]
        # get coordinates
        y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='white')
        # draw the box
        ax.add_patch(rect)
        # draw text and score in top left corner
        label = "%s (%.3f)" % (v_labels[i], v_scores[i])
        ax.text(x1, y1, label, color='white')

In [ ]:
# define the anchors
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]

# define the probability threshold for detected objects
class_threshold = 0.6
boxes = list()
for i in range(len(yhat)):
    # decode the output of the network
    boxes += utils.decode_netout(yhat[i][0], anchors[i], class_threshold, INPUT_H, INPUT_W)

# correct the sizes of the bounding boxes for the shape of the image
correct_yolo_boxes(boxes, image_h, image_w, INPUT_H, INPUT_W)

# suppress non-maximal boxes
utils.do_nms(boxes, 0.5)
# get the details of the detected objects
v_boxes, v_labels, v_scores = get_boxes(boxes, MSCOCO_LABELS, class_threshold)
# summarize what we found
for i in range(len(v_boxes)):
    print(v_labels[i], v_scores[i])

In [ ]:
# draw what we found
draw_boxes(photo_path, v_boxes, v_labels, v_scores)